In [2]:
import torch
torch.cuda.is_available()

True

In [4]:
import torchvision

In [13]:
torchvision.datasets.CelebA(root="./data/CelebA/", split="test", download=False)

RuntimeError: Dataset not found or corrupted. You can use download=True to download it